In [1]:
import matplotlib.pyplot as plt
import numpy as np

import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
print(tf.__version__)

2.3.1


In [5]:
training_set = tf.keras.preprocessing.image_dataset_from_directory(
    'D:/Docs/PCB/Training_Dataset',
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    seed=123,
    image_size=(110, 42),
    shuffle=True,
    validation_split = 0.2,
    subset="training"
)

Found 12000 files belonging to 8 classes.
Using 9600 files for training.
